In [1]:
print("Loading model and tokenizer...")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "microsoft/Phi-3-medium-128k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading model and tokenizer...


/DATA/rohan_kirti/miniconda3/envs/nilenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 6/6 [00:07<00:00,  1.31s/it]


In [2]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.7,
    "do_sample": True,
}

Device set to use cuda:0


In [5]:
def generate(sentence:str):
    prompt = f"""You are a Persuasion Strategy labeling expert.
    You are given an utterance.  
Your task is to identify and label the persuasion strategy it uses from the defined list of six strategies.  
You must also clearly explain why that strategy was chosen, and your explanation MUST be enclosed inside <reason> and </reason> tags exactly.  



 Guidelines for labeling:  
- Read the utterance carefully and determine its primary persuasion approach.  
- Select the persuasion strategy whose description aligns most closely with the utterance's content and intent.  
- When writing the reasoning, explain the specific features, keywords, or message style in the utterance that make it fit the chosen persuasion strategy.  
- The reasoning MUST be enclosed inside <reason> and </reason> tags — this is mandatory.  
- Be concise but clear, ensuring the explanation makes it obvious why that persuasion type applies.  



### Persuasion strategies:  

1. Logical appeal – Uses factual reasoning, statistics, verifiable claims, comparisons, or cost–benefit arguments to persuade.  
   - Even if one factual statement or claim is present (e.g., a price comparison, performance data, or measurable outcome), it qualifies as logical appeal, unless another strategy’s intent is more dominant.  
   - Examples: “This plan saves you $200 annually.” / “Our product is 25% more efficient than others.”  

2. Credibility appeal – Builds trust by emphasizing the trustworthiness, expertise, qualifications, or proven track record of the provider or source.  
   - This includes references to awards, rankings, certifications, industry recognition, or customer satisfaction ratings.  
   - Example indicators: “Rated 1 by…”, “Certified by…”, “Trusted by over 1 million customers.”  

3. Emotional appeal – Persuades by evoking emotions such as safety, comfort, fear, pride, hope, or peace of mind.  
   - Often uses reassuring or emotive language to create a personal emotional response rather than presenting objective facts.  
   - Example indicators: “Feel secure knowing…”, “Protect what matters most…”, “Imagine the joy of…”  

4. Personal appeal – Directly tailors the recommendation to the specific needs, concerns, or requests that the user has previously expressed.  
   - This strategy explicitly references what the user wants or has stated, making it a targeted recommendation.  
   - Example indicators: “Since you said you travel frequently…”, “Because you prefer low-maintenance options…”  

5. Persona appeal – Connects the recommendation to the user’s lifestyle, habits, values, or identity rather than a specific request.  
   - Often uses “if you are the kind of person who…” framing or aligns features with an implied personality type.  
   - Example indicators: “Perfect for adventure seekers…”, “If you enjoy luxury and comfort…”  

6. Default – Provides neutral, purely descriptive information without persuasive elements.  
   - No attempt to influence the decision is made; it is strictly informational.  
   - Example indicators: “Includes liability, collision, and comprehensive coverage.”  



Output format:  

Label: <strategy name>  
<reason>Reasoning for why this strategy was chosen, based on utterance features.</reason>  


Few-shot examples:  

Example 1:  
Sentence: "This plan saves you $200 annually compared to your current policy while offering the same coverage."  
Label: Logical appeal  
<reason>The plan presents a cost comparison and equivalent coverage, which is a fact-based logical argument.</reason>  

Example 2:  
Sentence: "Our company has been rated #1 in customer satisfaction for five years in a row."  
Label: Credibility appeal  
<reason>It highlights proven reputation and customer satisfaction history, which builds trust and credibility.</reason>  

Example 3:  
Sentence: "With this policy, you can relax knowing your family is protected in any situation."  
Label: Emotional appeal  
<reason>The focus is on feelings of security and reassurance, aiming to evoke emotional comfort.</reason>  

Example 4:  
Sentence: "Since you mentioned wanting full coverage for long road trips, this plan includes unlimited roadside assistance."  
Label: Personal appeal  
<reason>The recommendation is tailored to the user's stated need for long-trip coverage and roadside support.</reason>  

Example 5:  
Sentence: "If you enjoy weekend camping trips, this plan covers damage from off-road driving."  
Label: Persona appeal  
<reason>The plan connects to the user's lifestyle and activities, linking features to camping and off-road travel.</reason>  

Example 6:  
Sentence: "This policy covers liability, collision, and comprehensive damage."  
Label: Default  
<reason>The policy coverage is purely informative and describes coverage without persuasive elements.</reason>  

Now, identify the persuasion strategy and give the reasoning inside <reason> tags as instructed.  

Sentence: "{sentence}"  
Label:  
<reason></reason>  
"""

    messages = [
        {"role": "system", "content": "you are a helpful assistant. Label the utterance based on Persuasion Strategy and provide the reason"},
        {"role": "user",  "content": prompt},
    ]

    output = pipe(messages, **generation_args)
    response= output[0]['generated_text']
    return response.strip()


In [6]:
print(generate("Our comprehensive motor insurance covers accidents, theft, and damage, with premiums starting from just $1.20 a day — less than your morning coffee."))

Label: Logical appeal  
<reason>The statement gives a factual comparison between the cost of the insurance and something familiar like a morning coffee. It also outlines the coverage details, which are verifiable claims.</reason>


In [7]:
import pandas as pd
import os
import re

# Regex patterns to parse the extract() output
LABEL_RE = re.compile(r"Label\s*:\s*(.+?)(?:\s*(?:\n|$|<))", re.IGNORECASE | re.DOTALL)
REASON_RE = re.compile(r"<reason>(.*?)</reason>", re.IGNORECASE | re.DOTALL)

def parse_extraction(text: str) ->tuple:
    """Extract label and reason from the returned string."""
    if not text:
        return "", ""
    m_label = LABEL_RE.search(text)
    m_reason = REASON_RE.search(text)
    label = m_label.group(1).strip() if m_label else ""
    reason = m_reason.group(1).strip() if m_reason else ""
    return label, reason

def process_csv_with_pandas(input_path: str, output_path: str, start_from: int = 0):
    """
    Reads input CSV, processes each row, and writes to output CSV incrementally.
    Requires an extract(sentence) or generate(sentence) function in scope.

    Args:
        input_path: Path to input CSV
        output_path: Path to output CSV
        start_from: Row index (0-based) to start processing from
    """
    # Check if we are resuming or starting fresh
    if os.path.exists(output_path):
        processed_df = pd.read_csv(output_path)
        processed_count = len(processed_df)
    else:
        processed_df = pd.DataFrame()
        processed_count = 0

    # Read the whole input file
    df = pd.read_csv(input_path)
    if "new_agent_reply" not in df.columns:
        raise KeyError("Input CSV must contain a 'new_agent_reply' column.")

    # Determine actual starting point
    start_index = max(start_from, processed_count)

    # Process remaining rows one by one
    for idx in range(start_index, len(df)):
        sentence = str(df.loc[idx, "new_agent_reply"]).strip()
        result = generate(sentence)  # <-- your function here
        label, reason = parse_extraction(result)

        # Append label & reason to dataframe
        row = df.loc[[idx]].copy()
        row["label"] = label
        row["reason"] = reason

        # Append to the processed dataframe
        processed_df = pd.concat([processed_df, row], ignore_index=True)

        # Write back to CSV immediately so you can check progress
        processed_df.to_csv(output_path, index=False)
        os.sync()  # Force write to disk on some systems

    print(f"Processing complete. Output saved to {output_path}")



In [8]:
input_path="/DATA/rohan_kirti/niladri/Expert Datatset/Book1.csv"
output_path="/DATA/rohan_kirti/niladri/Expert Datatset/Engagement/results.csv"
process_csv_with_pandas(input_path, output_path, start_from=0)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing complete. Output saved to /DATA/rohan_kirti/niladri/Expert Datatset/Engagement/results.csv
